In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
import pickle
from transformers import BertTokenizer, BertModel
import torch
from numpy import ravel

### Drinks: 1
### Appetizers: 2
### Salads: 3
### Soups: 4
### Main: 5
### Desserts: 6

In [136]:
food_list_df = pd.read_csv('food_list_backup.csv').dropna().reset_index(drop = True)
food_list_df.Type = food_list_df.Type.astype('int32')
food_list_extra_df = pd.read_csv('food_list_extra.csv').rename(columns={"text": "Text", "category": "Type"})
food_list_tot_df = pd.concat([food_list_df, food_list_extra_df], axis=0).reset_index(drop = True)
food_list_tot_df.to_csv('food_list_tot.csv')

In [142]:
appetizers = ["Bruschetta", "Mozzarella Sticks", "Caprese Salad", "Calamari", "Oysters", "Shrimp Cocktail", "Guacamole", "Queso Fundido", "Egg Rolls", "Hummus", "Nachos", "Lettuce Wraps", "Jalapeno Poppers", "Stuffed Mushrooms", "Pigs in a Blanket", "Spinach Artichoke Dip", "Crockpot Buffalo Chicken Wings", "Cream Cheese Fruit Dip", "Cocktail Meatballs", "Hormel Chili Dip", "Mac and Cheese Bites", "Mozzarella Sticks", "Velveeta Sausage Dip", "Totchos", "Cowboy Crack Dip", "Pizza Monkey Bread", "Easy Bacon-Wrapped Shrimp", "Buffalo Chicken Dip", "Pizza Rolls", "Extra Crispy Onion Rings", "Cheeseburger Sliders", "Deviled Eggs", "Quick & Easy Nachos", "Southern Hush Puppies", "Caramelized Onion Dip", "Homemade Bloomin’ Onion", "Mini-Bagel Pizzas", "Soft Pretzel Bites", "Stuffed Jalapeños", "Fried Pickles", "Potato Skins", "Quesadillas", "Garlic Bread", "Chips and Salsa"]
salads = ["Easy Nicoise Salad", "Vegan Roasted Sweet Potato Salad", "Baked Salmon with Grapefruit Salad", "Curried Cauliflower Quinoa Salad", "Kale Caesar Salad", "Tropical Kale Salad", "Classic Cobb Salad with Buttermilk Dressing", "Southwest Style Cobb Salad with Smoky Chipotle Dressing", "Spring Cobb Salad", "Grilled Curry Chicken Salad with Herb Vinaigrette", "Chicken Waldorf Salad", "Curried Chicken Salad Salad", "Simple Butter Lettuce Salad", "Mediterranean Wedge Salad", "Simple Italian Salad", "Blackberry Goat Cheese Salad", "Tomato Salad", "Cucumber Salad", "Grilled Romaine Caesar", "Watermelon Cucumber Salad", "Maple Roasted Squash Salad", "Celery Pear and Hazelnut Salad", "Shaved Brussels Sprout Salad with Creamy Maple Dressing", "Roasted Beet Salad with Chèvre Green Goddess Dressing", "Red Cabbage and Apple Slaw", "Asian Slaw With Sriracha Dressing", "Radish Salad with Yogurt Dill Dressing"]
soups = ["Aguadito", "Ajiaco", "Avgolemono", "Bak kut teh", "Bánh canh", "Borscht", "Bouillabaisse", "Bún bò Huế", "Bún riêu", "Caldo verde", "Canh chua", "Cazuela", "Changua", "Chicken noodle soup", "Chupe", "Clam chowder", "Cock-a-leekie soup", "Egg drop soup", "Fanesca", "French onion soup", "Gazpacho", "Goulash soup", "Gumbo", "Harira", "Hot and sour soup", "Kharcho", "Kimchi jjigae", "Laksa", "Lentil soup", "Miso soup", "Mulligatawny", "Phở", "Pozole", "Ramen", "Samgyetang", "Solyanka", "Tom kha kai", "Tom yum", "Vichyssoise", "Zuppa toscana"]
main = ["Biryani", "Ceviche", "Dim Sum", "Sushi", "Ramen", "Satay", "Pho", "Peking Duck", "Fajitas", "Paella", "Ratatouille", "Bratwurst", "Pierogi", "Poutine", "Hamburger", "Hot Dog", "Pizza", "Pasta", "Masala Dosa", "Butter Chicken", "Tandoori Chicken", "Fish and Chips", "Shepherd's Pie", "Bangers and Mash", "Lobster Roll", "Clam Chowder", "Barbecue Ribs", "Jerk Chicken", "Empanada", "Feijoada", "Arepas", "Tamales", "Goulash", "Moussaka", "Tagine", "Pad Thai", "Nasi Goreng", "Lamb Kebabs", "Chicken Parmesan", "Beef Wellington"]


df_appetizers = pd.DataFrame({'Text':appetizers,'Type':2*np.ones(len(appetizers))})
df_appetizers.Type = df_appetizers.Type.astype('int32')
df_salads = pd.DataFrame({'Text':salads,'Type':3*np.ones(len(salads))})
df_salads.Type = df_salads.Type.astype('int32')
df_soups = pd.DataFrame({'Text':soups,'Type':4*np.ones(len(soups))})
df_soups.Type = df_soups.Type.astype('int32')
df_main = pd.DataFrame({'Text':main,'Type':5*np.ones(len(main))})
df_main.Type = df_main.Type.astype('int32')

food_list_tot_df = pd.concat([food_list_tot_df, df_appetizers, df_salads, df_soups, df_main])
food_list_tot_df = food_list_tot_df[~food_list_tot_df['Text'].str.lower().duplicated(keep='first') & food_list_tot_df['Text'].str.replace(' ', '').str.isalpha()].reset_index(drop=True)

food_list_tot_df.to_csv('food_list_combined.csv', index = False)

In [3]:
food_list_tot_df = pd.read_csv('food_list_combined_old.csv').dropna().reset_index(drop = True)
food_list_tot_df.Type = food_list_tot_df.Type.astype('int32')

In [4]:
food_list_tot_df[food_list_tot_df.Type == 0]#.sort_values(by = 'Text')

,Text,Type
399,Add Chips and a Cookie,0
401,Add a Drink,0
402,Add Fruit,0
754,Choice of three,0


## BERT Conversion

In [6]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def texts_to_vectors(texts):
    print("Starting texts_to_vectors function...")
    
    vectors = []
    for cnt, text in enumerate(texts):
        try:            
            inputs = tokenizer([text], return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
            mask = inputs.attention_mask
            masked_embeddings = embeddings * mask.unsqueeze(-1)
            summed = torch.sum(masked_embeddings, 1)
            summed_mask = torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = summed / summed_mask.unsqueeze(-1)
            vectors.append(mean_pooled[0].numpy())
        except Exception as e:
            print(f"Error encountered while processing text {cnt}: {e}")
            continue
        # Print progress update every 400 samples
        if cnt % 400 == 0:
            percentage = (cnt / len(texts)) * 100
            print(f"Processing text {cnt} of {len(texts)} ({percentage:.2f}% complete)")
    
    print("Finished converting texts to vectors.")
    return np.array(vectors)

#vectors = texts_to_vectors(food_list_tot_df['Text'].tolist())
vectors = texts_to_vectors(food_list_tot_df['Text'].str.lower().tolist())

df_vectors = pd.DataFrame(vectors, columns=[f'vector_{i}' for i in range(vectors.shape[1])])
df_bert_test = pd.concat([food_list_tot_df, df_vectors], axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting texts_to_vectors function...
Processing text 0 of 755 (0.00% complete)
Processing text 400 of 755 (52.98% complete)
Finished converting texts to vectors.


In [7]:
#df_bert_test.shape
df_bert_test2 = df_bert_test.copy()

In [183]:
df_bert_test = df_bert_test2.copy()

In [8]:
# Create a new column 'salad' that has a value of 1 if 'text' contains the word 'salad', and 0 otherwise

words = ['salad','soup','chowder', 'appetizer', 'fries','strip','bowl', 'chips', 'steak', 'platter', 'pudding',\
         'chocolate','malt', 'shake','cream','creme','vanilla','brownie', 'pie', 'rings', 'wrap',\
         'juice', 'coffee', 'milk', 'tea', 'bites','drink','orange','water','burger','meat','nacho','sandwich',\
        'patty','tater','burrito','skillet','lattte','esspresso', 'cafe','sausage', 'ice cream','beer','wine','ice',\
        'caesar','cola','coke','slider','broth','pasta','shrimp','cookie','sundae','signature','choice','latte','mocha'] 
# recently added: 'cookie','sundae'
# recently added: 'signature','choice','latte','mocha'

for word in words:
    df_bert_test[word] = df_bert_test['Text'].str.contains(word, case=False).astype(int)

words2 = words.copy()
words2.extend(['Text','Type'])
df_bert_test[words2][:100]

,salad,soup,chowder,appetizer,fries,strip,bowl,chips,steak,platter,pudding,chocolate,malt,shake,cream,creme,vanilla,brownie,pie,rings,wrap,juice,coffee,milk,tea,bites,drink,orange,water,burger,meat,nacho,sandwich,patty,tater,burrito,skillet,lattte,esspresso,cafe,sausage,ice cream,beer,wine,ice,caesar,cola,coke,slider,broth,pasta,shrimp,cookie,sundae,signature,choice,latte,mocha,Text,Type
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fresh Fruit,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bagel,2
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fruit,2
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Soft Tofu Soup,4
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Noodle Soups,4
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Rice Noodle Soup,4
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bamee Wonton,4
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Chicory Salad,3
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Golden Beets,3
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Cajeta Ice Cream,6


## Train Test Split & XGBoost

In [9]:
X = df_bert_test.drop(['Text','Type'],axis = 1)
y = df_bert_test[['Type']]

test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=0)

# xgb = XGBClassifier(n_estimators= 150,learning_rate = 0.3,random_state=0)
# xgb.fit(X_train, y_train)
# score = xgb.score(X_test, y_test)
# y_hat_tree = xgb.predict(X_test)

# print('XGBoost Score: ', score)

In [10]:
xgb = XGBClassifier(n_estimators= 150,learning_rate = 0.3,random_state=0)
xgb.fit(X_train, y_train)
score = xgb.score(X_test, y_test)
y_hat_tree = xgb.predict(X_test)

print('XGBoost Score: ', score)

XGBoost Score:  0.7136563876651982


In [9]:
# xgb = GradientBoostingClassifier(n_estimators= 150,learning_rate = 0.3,max_depth = 8,random_state=0)
# # 150, 3 : 67.355%
# y_train_GBC = ravel(y_train)
# xgb.fit(X_train, y_train_GBC)
# y_test_GBC = ravel(y_test)
# score = xgb.score(X_test, y_test_GBC)
# y_hat_tree = xgb.predict(X_test)

# print('GradientBoostingClassifier Score: ', score)
# #.6564

GradientBoostingClassifier Score:  0.6299559471365639


In [11]:
with open('/Users/alex/Data_Science/Menu_App/menu_function/MyFunction/xgb2_5.pkl', 'wb') as f:
    pickle.dump(xgb, f)

In [11]:
len(words)+768

816

In [12]:
df_bert_test.shape

(806, 818)

In [ ]:
xgb = XGBClassifier(random_state=0)


param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 4, 6, 8],
    'learning_rate': [0.01,0.05, 0.1, 0.2]
}

grid_search = GridSearchCV(xgb, param_grid, cv=3, scoring='accuracy')

# Fit the RandomizedSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

In [88]:
rfc = RandomForestClassifier(bootstrap=True, n_estimators=600,random_state=0)
rfc.fit(X_train, y_train)
score = rfc.score(X_test, y_test)
y_hat_tree = rfc.predict(X_test)

print('Score: ', score)

/var/folders/mq/z3r_z9nd59g6z5rfmlxsltg40000gn/T/ipykernel_25813/4208886092.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


Score:  0.6024844720496895


In [10]:
items = ['COFFEE', 'CAFE LATTE', 'BLOOD ORANGE', 'MIMOSA', 'ESPRESSO', 'MOCHA', 'BUTTER FRENCH TOAST', 'BRAISED OXTAIL TAQUITOS HUEVOS', 'PORCHETTA EGGS BENEDICT', 'BREAKFAST SANDWICH', 'EGG WHITE OMELETTE', 'RIBEYE', 'HASH SKILLET', 'HAND ROLLED CINNAMON ROLL', 'BAGEL', 'BREAKFAST POTATOES', 'APPLEWOOD SMOKED BACON', 'FRUIT', 'BUTCHER LARDER', 'YOGURT PARFAIT', 'SAUSAGE']
items4 = ['STARTER','GOLDEN BEETS','CHICKEN FRIED CHICKEN','MUSHROOM ENCHILADAS','CAJETA ICE CREAM']
items2 = ['Greek Salad S', 'Aubrees Wings SO', 'Caesar Salad', 'Bacon Brussels Sprouts SO', 'Garden Salad S', 'AUBREES SIGNATURE PIZZAS', 'BBQ Chicken', 'The Ultimate Feta Bread', 'Mac  Cheese', 'Cheese Bread S', 'Pasta Gone Bayou', 'Aubrees Bread', 'MARGHERITA', 'Spaghetti  Meatball', 'Lasagna Rolls', 'Aubrees Chili quart SOGF', 'Tomato Bisque quart']

In [30]:
test = pd.DataFrame({'Text':items2, 'Type': np.zeros(len(items2),dtype = int).tolist()})

vectors = texts_to_vectors(test['Text'].tolist())
df_vectors = pd.DataFrame(vectors, columns=[f'vector_{i}' for i in range(vectors.shape[1])])
df_bert_test2 = pd.concat([test, df_vectors], axis=1)

for word in words:
    df_bert_test2[word] = test['Text'].str.contains(word, case=False).astype(int)

df_predict = df_bert_test2.drop(['Text','Type'],axis = 1)
Type = xgb.predict(df_predict)
df_bert_test2['Type'] = Type
df_bert_test2[['Text','Type']]

Starting texts_to_vectors function...
Processing text 0 of 17 (0.00% complete)
Finished converting texts to vectors.


,Text,Type
0,Greek Salad S,3
1,Aubrees Wings SO,5
2,Caesar Salad,3
3,Bacon Brussels Sprouts SO,5
4,Garden Salad S,3
5,AUBREES SIGNATURE PIZZAS,5
6,BBQ Chicken,5
7,The Ultimate Feta Bread,5
8,Mac Cheese,5
9,Cheese Bread S,5
